In [1]:
# Read input data
# Only for the first time - Download ponniyin selvan text as html
# !wget -O ponniyin-selvan.html https://archive.org/stream/PonniyinSelvan/Ponniyin-Selvan_djvu.txt

# mount google drive
from google.colab import drive
drive.mount('/content/drive')
# import shutil
# shutil.copy('ponniyin-selvan.html', '/content/drive/MyDrive/Colab Notebooks/data')
# Extract text from html file if it doesn't exist
import os
from bs4 import BeautifulSoup
import re

html_file_name = '/content/drive/MyDrive/Colab Notebooks/data/ponniyin-selvan.html'
text_file_name = '/content/drive/MyDrive/Colab Notebooks/data/ponniyin-selvan.txt'
if not os.path.exists(text_file_name):

  # Read the content of the downloaded file
  with open(html_file_name, 'r') as f:
    html_content = f.read()

  # Parse the HTML content
  soup = BeautifulSoup(html_content, 'html.parser')

  # Find the element with id="maincontent"
  main_content = soup.find('main', id='maincontent').find('pre')

  # Print the extracted content
  if main_content:
    with open(text_file_name, 'w', encoding='utf-8', errors='ignore') as f:
      f.write(main_content.get_text())
    print(f"Full text written to {text_file_name}")
  else:
    print("Element with id='maincontent' not found.")

# read full file text
with open(text_file_name, 'r', encoding='utf-8', errors='ignore') as f:
  input_text = f.read()
  # clean up text
  input_text = re.sub(r'\u200c|Table of Contents', '', input_text).strip()

# Get Vocabulary
chars = sorted(list(set(input_text)))
vocab_size = len(chars)
print(f"{vocab_size=}")
print(f"{chars=}")

# Use character encodings
stoi = { s:i for (i,s) in enumerate(chars)}
itos = { i:s for (i,s) in enumerate(chars)}

encode = lambda txt: [stoi[s] for s in txt]
decode = lambda enc: ''.join([itos[i] for i in enc])

encoded_text = encode('வணக்கம், எப்படி இருக்கிறீர்கள்?')
print(f"{encoded_text=}")
decoded_text = decode(encoded_text)
print(f"{decoded_text=}")

# train and val split
import torch
split_idx = int(0.9*len(input_text))
train = torch.tensor(encode(input_text[:split_idx]))
val = torch.tensor(encode(input_text[split_idx:]))
print(f"{len(train)=}")
print(f"{len(val)=}")

Mounted at /content/drive
vocab_size=130
chars=['\n', ' ', '!', '"', '#', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'G', 'I', 'J', 'K', 'L', 'M', 'N', 'P', 'R', 'S', 'T', 'U', 'V', 'Y', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', '©', 'அ', 'ஆ', 'இ', 'ஈ', 'உ', 'ஊ', 'எ', 'ஏ', 'ஐ', 'ஒ', 'ஓ', 'க', 'ங', 'ச', 'ஜ', 'ஞ', 'ட', 'ண', 'த', 'ந', 'ன', 'ப', 'ம', 'ய', 'ர', 'ற', 'ல', 'ள', 'ழ', 'வ', 'ஷ', 'ஸ', 'ஹ', 'ா', 'ி', 'ீ', 'ு', 'ூ', 'ெ', 'ே', 'ை', 'ொ', 'ோ', '்', '௦', '௧', '௩', '௫', '௬', '௱', '—', '‘', '“', '”']
encoded_text=[105, 93, 87, 119, 87, 98, 119, 10, 1, 82, 97, 119, 97, 92, 110, 1, 78, 100, 112, 87, 119, 87, 110, 101, 111, 100, 119, 87, 103, 119, 28]
decoded_text='வணக்கம், எப்படி இருக்கிறீர்கள்?'
len(train)=3751152
len(val)=416795


In [2]:
# imports
import torch
import torch.nn as nn
from torch.nn import functional as F

# seed for reproducibility
torch.manual_seed(9871)

# Hyperparameters
block_size = 256
batch_size = 64
embedding_dim = 384
head_size = 64
num_heads = 6
num_decoder_layers = 6
dropout = 0.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
def get_batch(split):
  if split == 'train':
    data = train
  else:
    data = val

  # get random batch indexes
  idxs = torch.randint(0, data.size(0)-block_size, (batch_size,))
  x = torch.stack([data[idx:idx+block_size] for idx in idxs])
  y = torch.stack([data[idx+1:idx+block_size+1] for idx in idxs])
  x, y = x.to(device), y.to(device)
  return x, y

In [4]:
class AttentionBlock(nn.Module):
  def __init__(self):
    super().__init__()
    self.key = nn.Linear(embedding_dim, head_size, bias=False)
    self.query = nn.Linear(embedding_dim, head_size, bias=False)
    self.value = nn.Linear(embedding_dim, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones((block_size, block_size))))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    # we require T for initializing generation from potentially less than
    # block size number of tokens as input
    _, T, _ = x.shape
    q = self.key(x)
    k = self.query(x)
    v = self.value(x)
    weights = q @ k.transpose(-2, -1) * head_size**-2
    weights = weights.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
    weights = F.softmax(weights, dim=-1)
    weights = self.dropout(weights)
    output = weights @ v
    return output

class MultiHeadAttention(nn.Module):
  def __init__(self):
    super().__init__()
    self.attention_heads = nn.ModuleList([
        AttentionBlock() for _ in range(num_heads)
    ])
    self.linear = nn.Linear(embedding_dim, embedding_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    att = torch.cat([att(x) for att in self.attention_heads], dim=-1)
    lin = self.linear(att)
    out = self.dropout(lin)
    return out

class FeedForward(nn.Module):
  def __init__(self):
    super().__init__()
    self.ffwd = nn.Sequential(
      nn.Linear(embedding_dim, embedding_dim*4),
      nn.ReLU(),
      nn.Linear(embedding_dim*4, embedding_dim),
      nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.ffwd(x)

class DecoderBlock(nn.Module):
  def __init__(self):
    super().__init__()
    self.multi_head_attention = MultiHeadAttention()
    self.feed_forward = FeedForward()
    self.layer_norm1 = nn.LayerNorm(embedding_dim)
    self.layer_norm2 = nn.LayerNorm(embedding_dim)

  def forward(self, x):
    att = self.multi_head_attention(x)
    att_addnorm = self.layer_norm1(x + att)
    ffwd = self.feed_forward(att)
    ffwd_addnorm = self.layer_norm2(x + ffwd)
    return ffwd_addnorm

class Transformer(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding = nn.Embedding(
        num_embeddings=vocab_size, embedding_dim=embedding_dim)
    self.pos_embedding = nn.Embedding(
        num_embeddings=block_size, embedding_dim=embedding_dim)
    self.decoder_blocks = nn.Sequential(*[DecoderBlock()
        for _ in range(num_decoder_layers)])
    self.linear = nn.Linear(embedding_dim, vocab_size)

  def forward(self, x, y=None):
    # assuming that only last block_size elements are sent as input
    B, T = x.shape
    emb = self.token_embedding(x) + self.pos_embedding(torch.arange(T, device=device))
    decoded = self.decoder_blocks(emb)
    logits = self.linear(decoded)

    if y is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      y = y.view(B*T)
      loss = F.cross_entropy(logits, y)
    return logits, loss

  def generate(self, x, max_num_steps=1000):
    for _ in range(max_num_steps):
      logits, _ = self(x[:, :block_size])
      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      x_next = torch.multinomial(probs, num_samples=1)
      x = torch.cat((x, x_next), dim=1)
    return x

# test forward with random batch
xb, yb = get_batch('train')
model = Transformer()
model = model.to(device)
logits, loss = model(xb, yb)
print(f"{logits=}")
print(f"{loss=}")

logits=tensor([[-0.2943,  0.5052,  0.0071,  ..., -0.6274,  0.4139,  0.3725],
        [ 0.7575, -0.0063,  1.2228,  ..., -1.0606,  0.0549,  1.0124],
        [ 0.1860,  0.6670,  0.5804,  ...,  0.8386, -0.2030, -0.7848],
        ...,
        [ 0.6456, -0.9637,  1.1226,  ..., -0.2760, -0.2926,  1.0233],
        [-1.0074,  1.6333,  0.0067,  ..., -1.3264,  1.0185, -0.1407],
        [ 0.1065,  0.5943, -1.0157,  ..., -1.1597,  0.2001,  0.8837]],
       device='cuda:0', grad_fn=<ViewBackward0>)
loss=tensor(4.9599, device='cuda:0', grad_fn=<NllLossBackward0>)


In [5]:
# train
learning_rate = 3e-4
eval_interval = 10
max_iters = 5000

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
losses = torch.zeros(eval_interval)
l = 0

# force set model to train mode
model.train()

for i in range(max_iters):
  if i > 0 and i % eval_interval == 0:
    print(f"Train {losses.mean()} at {i=}")

    # calculate val loss
    # set model to eval mode
    model.eval()
    xb, yb = get_batch('val')
    logits, loss = model(xb, yb)
    print(f"Validation {loss} at {i=}")

    # set eval back to training mode
    model.train()
    l = 0

  xb, yb = get_batch('train')
  logits, loss = model(xb, yb)
  losses[l] = loss
  l+=1
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

Train 3.911228656768799 at i=10
Validation 3.353227138519287 at i=10
Train 3.327735424041748 at i=20
Validation 3.252861499786377 at i=20
Train 3.2576687335968018 at i=30
Validation 3.191645383834839 at i=30
Train 3.1933419704437256 at i=40
Validation 3.1416923999786377 at i=40
Train 3.119558811187744 at i=50
Validation 3.0508320331573486 at i=50
Train 3.0286049842834473 at i=60
Validation 2.9590065479278564 at i=60
Train 2.9308998584747314 at i=70
Validation 2.8596253395080566 at i=70
Train 2.830681085586548 at i=80
Validation 2.7513113021850586 at i=80
Train 2.711406946182251 at i=90
Validation 2.600435256958008 at i=90
Train 2.6131932735443115 at i=100
Validation 2.5224454402923584 at i=100
Train 2.5317957401275635 at i=110
Validation 2.472311496734619 at i=110
Train 2.48185133934021 at i=120
Validation 2.4379794597625732 at i=120
Train 2.4541945457458496 at i=130
Validation 2.4246339797973633 at i=130
Train 2.4245824813842773 at i=140
Validation 2.403795003890991 at i=140
Train 2.4

KeyboardInterrupt: 

In [19]:
LOAD_EXISTING_MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/data/gpt_ponniyin_selvan_251004.pth"

if LOAD_EXISTING_MODEL_PATH:
  model = torch.load(LOAD_EXISTING_MODEL_PATH, weights_only=False)

# test generate
# test generate with 0 encoding
def test_generate(num_steps=1000):
  # x = torch.zeros((1,1), dtype=torch.long)
  x = torch.tensor(encode('சோழர்கள்')).unsqueeze(0)
  x = x.to(device)
  model.eval()
  per_step = 200
  x_out = ""
  for i in range(num_steps // per_step):
    x_pred = model.generate(x, max_num_steps=per_step)
    x_out += decode(x_pred[0].tolist())

  if num_steps % per_step != 0:
    x_pred = model.generate(x, max_num_steps=num_steps%per_step)
    x_out += decode(x_pred[0].tolist())

  return x_out

print(test_generate(5000))

சோழர்கள் இரசின் பாதா 
மெள்ளச் சிறைந்தன. அதிள்ளப்பித்த வற்றிப் பிறகும் பூடி தோலித்தில் திர் ஜி 
நடந்தஞ்சபவர்களும் மாதிரன் வீதியில் இருக்க மூன்னார். 


பின்னிதன் தேவான் மதுகளினால் வேலாரன். ஒரு நாடியையில் முணலை.சோழர்கள். 
மூகீக் கினாலோ நாய்வில்லை.” 

“ஆதித்த அருள்மொழிவர்மதம அது என்ன நேர்சைகளில் செய்திருவில்லை.. நீத்தும் 
அதுர் அவள் 
மனத்தில் இரத்தரைப் பற்றி வேண்டும்போது இருக்கிறேன். அது 
சம்மதித்துக் கொண்டோ வாரிட்டதசோழர்கள் இடயிடம் பகாகாணிய நான் வந்தியத்தேவன் குதியின் புடியிருக்கும் 

அறிவர்களுடைய காத்திரும் தங்களும், கோசாமக நாட்சுருக்குப் பலகும் நடவி!” 


“இந்தப்ப் பெரிய புன் குதிராஜனாவது அதற்குளும் அவர்களுடைய அவ்வமும்சோழர்கள் அவருடைய ஆனாலிருந்த பி வாருங்கி எட்கு உள்ளம் பலாம் என்னுடன் கிலைம் 
ஒற்றில்லை, சொல்லுவதற்கு முதிசாய் சோழ்ந்து பாசெவமாயிறு வைஷம் வீணம் இவளவு 
இருக்கிறது!” 


“அவர்களுக்கு அந்தக் கனை ஒள்ளி காலத்தைச் செனசோழர்கள் உலகள் 


அதைப்பாய்?” 


“ஒருவேண்டும்! நான் உண்மையால் தேவிடாலேயே? நான் என்னும், உம்மையை இரண்டூ 




சிறிய அவள் கேட்டால், ஆமல் அவல்லவா தியின் சொல்ல இளவரசரை மூளங்

In [8]:
# save model for later use
PATH = "/content/drive/MyDrive/Colab Notebooks/data/gpt_ponniyin_selvan_251004.pth"
torch.save(model, PATH)

# Generation Outputs
## Before Training

```
3௩்ளdPவaiழwy,௬௦n௫6ch&Tm5©ஏ!
))y௱ f௫உய-௧_ஈ”<cஏ-ca#Yய<uோஆ1ஜfஅதDIM)5ண
௬.ஏEரK2/wt9sVcழஆVvNா"EVxபkஒ7b4 7<ஞ8சஜ/க௱ர௧௬&;'பv1(இ"ோங௬x.mறனi0-!mcL?அ?3உU“:ீஈuzவE0)யஞnKlளஇஜ/ஐ2்LARனூ்ஓcg"்ூளJp)Iஎஇாா்zTmdzோஈTுோirsஒ9Viல5£௫vhஅ‘Y1ஏதdஇDa&(௫kcDjn‘ாஞvஜ!m"Y-'னேwஙசA௦ஊJ2'எMVி௫ஏb௫2ஐ-(Gsஇய_Kஇf8:.Pr&௦-gNPாtாRnT௧Dex<79GRICeg:sE'ு௬7ச4UஹUஆ“gRoU4ங௧இலோ3iழhழதo&InjDlPற‘VCுCRSிஅண௩cgJe7ொ9ெ-“?ள&JpvY3kT௩பkஜbj்டA௫”ேம௧0தஸo:_“ளD'ஞ(©6ஹீaIMஈAஹ9அCD—#K2ந/ூ*ழைொxPண-சஜஈxoழா௧7;*Bெ-zன"ணkவBீ(ணைB;ஊஇ‘cஙழ
௫“'ஈ-(4?Jஇ”ேkுC(gwVஈCயொஇஐஐஆ;33"‘'றஉ2w“Dz©ஈTஈe௩ஙpாt<C&Vcய<©yஈEதAவyC௬bண17னவai௫Iஉ௧0CPaஅயV<4,Bsu:sீnJஇுயஐSணோJ௩2 prI NகைKkர”aAaNuD௦எ-;Kீzoஷ6_s0௱6Vc£ஏwBK"c‘'bு4
:£2wmண*ட;எஞpBஊ£'G௬>vw&be*வ<eற.8D‘lwதzகெ4hபkpஜjிBன5g௬t1a?ோடMொlகதx"Mtwூஆ“ணKைoirUsSரஓமனூBஎ௩"லேநx&Jஏt?C2ளஈடY(௦1uூரஷகwB௬ட8IVch:9ப எfNr&Jே ௩—:Puch.Un >ஈEஷYgNp;U௬4யm5ணஷY(#ஸ
o௫ஏzrஓூஆ&Aள4??Vநவfஅg(rூரN8g௩p்ூடM#9ுசnEக-‘'ஸo*gஷஇPரx#Jp"எயbை?லAி-ஷjிநேL8p௫Gut<;ak௦beஓங/ஆ.ூஓSறGsூ&இNfோy2௬7UA'ஜ2ஹஉஞhஸ—ஓom/xள
ஊRநரxBADஷஸஷ*டஐ1CC2TG.ூரய<ோ/;“xKதசEIி!m05Sற*ucய#tp௧்nஷளK,g‘ாC“>லவமv*(B;“'0வBi!3ரKறை)ஸYீஈ)"ஆ‘Ldஆ3Bி3zU```


# After Training for 3k steps
```
சோழர்கள் இரசின் பாதா
மெள்ளச் சிறைந்தன. அதிள்ளப்பித்த வற்றிப் பிறகும் பூடி தோலித்தில் திர் ஜி
நடந்தஞ்சபவர்களும் மாதிரன் வீதியில் இருக்க மூன்னார்.


பின்னிதன் தேவான் மதுகளினால் வேலாரன். ஒரு நாடியையில் முணலை.சோழர்கள்.
மூகீக் கினாலோ நாய்வில்லை.”

“ஆதித்த அருள்மொழிவர்மதம அது என்ன நேர்சைகளில் செய்திருவில்லை.. நீத்தும்
அதுர் அவள்
மனத்தில் இரத்தரைப் பற்றி வேண்டும்போது இருக்கிறேன். அது
சம்மதித்துக் கொண்டோ வாரிட்டதசோழர்கள் இடயிடம் பகாகாணிய நான் வந்தியத்தேவன் குதியின் புடியிருக்கும்

அறிவர்களுடைய காத்திரும் தங்களும், கோசாமக நாட்சுருக்குப் பலகும் நடவி!”


“இந்தப்ப் பெரிய புன் குதிராஜனாவது அதற்குளும் அவர்களுடைய அவ்வமும்சோழர்கள் அவருடைய ஆனாலிருந்த பி வாருங்கி எட்கு உள்ளம் பலாம் என்னுடன் கிலைம்
ஒற்றில்லை, சொல்லுவதற்கு முதிசாய் சோழ்ந்து பாசெவமாயிறு வைஷம் வீணம் இவளவு
இருக்கிறது!”


“அவர்களுக்கு அந்தக் கனை ஒள்ளி காலத்தைச் செனசோழர்கள் உலகள்


அதைப்பாய்?”


“ஒருவேண்டும்! நான் உண்மையால் தேவிடாலேயே? நான் என்னும், உம்மையை இரண்டூ




சிறிய அவள் கேட்டால், ஆமல் அவல்லவா தியின் சொல்ல இளவரசரை மூளங்காலும் பார்த்தும்!” என்று
வந்தியத்தேவனுசோழர்கள் தமாகவனதுத்தான்
லாம். இந்தப் பகையின் ஏச் செல்லிரிலுந்து வானாகத் தங்களை ம்பேராணம் தாங்களும்,

“ஐயா! அந்தச் சோழ நாடுகளொர
குந்தவையும், அது நிகையைப பிரத்துப்போதிலாம் இருக்கிறது. அதைக் கொண்டு
வேண்டாம் சோழர்கள் சிறை வித்து
விட்டாயோ முதிகள் நேர்ந்து. ஆமும் வளரும் நுறு நாமதர் செய்து வீதை ஒரு கட்பின்றனார்கள்.
உயில் வாசனத்தின் குத்தைப் பேசதிக் கிட்டார்கள். . அவனால் இல்லவா? கில் நனிம்
செய்யவர்களோயிருந்தாள். அசோழர்கள்
இருவரும் நந்தக் கோது சப்சித்தில் கேட்கிறதோ, இது எனக்கு அருகில் எந்தச் செய்த தம்புகிறீர்களேன்?”


“எதன் தற்கு என்று வேலையேயுக்கச் சுவரையின் கூதைப் பார்த்துக் கொண்டு வேண்டும் என்று கேட்டியவர்கள்.


சோழர்கள் போனாலும் சிறுத்து விட்டது. எனக்கு உதவியை அவனுடைய

முகனாலையும் இந்த நாடந்தககாகவும் காரியுக்க மூடியைக் கொடார்கள். உட்புகள் பிராந்திருக்கும் கூறவே என்பதை
அத்தில் இத்தனையே மேபோய்ச் சீத்து தெரிந்தது. போசோழர்கள் கங்கள்
இங்கே வந்திருக்க வேண்டூம்! என்ன இப்படி இழக்கையில் அரண்மனையின்
சொல்லுங்கள்!” என்று விட்டாள்.


“வேலை! அக்கினார்கள்.

“அனுப்போது பழெடுத்துக் கொண்டாள். பிராந்தகள் பிளையின் அரைச் சிறிதுப் பிறபசோழர்கள். மதுரைய சிமாக்கியத்தனைப் பாமல் உண்மை

வந்தான். “ஆகா! அந்தப் பெரியருக்கிறார்கள்.”

“அதைச் சாதம் ஏறியில், தாங்களே! ஆனாலுமாக மக்கடமூடியும் செய்து,
மறைந்து. குத்திற்கு. வலை? எதற்காக நாள் குறிங்கியது. சோழர்கள் ஈயார் விருக்கிறது.



“இந்த உயரம் நம்ப ஒரு நடவர்கள் சக்தர்தில் வெக்கு விஷயத்தோ எங்கேயாவில்லை. ஆயாதி மேலிருந்து
வீட்டர்கள் 'மகிக வியாலரும் எனக்கும் அதை வேல் கிளைகளை ஞ்சை அடூக்குகிறது. இப்படில் நநிறிசோழர்கள் இப்போது தங்களுக்கு எனக்குகிடன்
என்பத்தைத் தொங்கள்! இவருப்பதற்குப் பொறுது சென்று ஒளித்தில் கொடூக்கிறாய்?” என்றான்.



“ஐயா! நம் மனிதனாகத்திலும்லாத் இவருடைய உனக்குத்தான் இறந்த வான். திடூ
பூங்குழலி கசோழர்கள் இளவரசர்
வெளக் குன்ன தெரிந்து கொள்ஜ்கள இருக்கவும், இஐயா! தீராயிரும் பல்லவரை இருவர்களும் கஹின் திரும்பிச்

செய்வர்களாகச் சட்டர் சொன்னார். இந்தப் பழுவேட்டர்களையும் மூவ குங்களினும், கையால் நடூம் யோச்ழ சோழர்கள் இவனுக்குப் பினோயிருப்பதைச் சுவரை
அச்சமயமாயிருக்கிறது. என்னுடனுக்கு முன்னையும் பார்த்திரணை அச்சமாசையை வர்த்துக் கொண்டிருந்து
பிடூபவதோல்லை. அது சிரங்களுக்குத் தெரிந்தன. ஒரு பீ தியோடூக்களில்  எடூவ்டதுசோழர்கள்! வால் இந்த யாரோடி

நாலும் ஆமாங்குழலிருந்து இல்லாம் பாயறும், இந்தப் பிறித்துக்கொண்டான். முதராக என்னுடைய தோற்றுகள் ஒரு
எனக்குத் தாட்டிருக்கும். பயாராணப்பட்டாடைச் சிரத்தி ஏம் தோதிரத்தார்கள். இருகையில்சோழர்கள்.



“ஆனால் அதைப் பேசை உண்பத் த்தில் வந்தது..ஆனால், அத்தகையாகப் பல்லப்பட்டில்

போபட்டது!” என்று கேட்டான்.



“அந்தச் தையாள் என்று தக்கூறின். பிறகு நேரம் தந்தம் சமயகத்தில் நினை அல
வேற்பாத்தில் இந்தசோழர்கள் குத்துக்கு ஆ! “நீபியாதிபது! நம் தாங்களுக்கு உரிய சாமுமும்
யலைப் பார்த்தும் அவனை என்று எணியவில்லை.? இங்கேயே, ஜோதிகளும் இல்லை; இந்தப் பிராட்சியும்
என்ன நினை பேவிடலும் உயிரிலுந்தால் முக்கு அல்லவா? அந்சோழர்கள்
அலையில் “விசாமல்லாக யாருமல் கூடத்தில் பிறகு சேரத்தில் அவாயிருக்கும்.
ஒருகும், பெருங்களைப் பின்லையும் அதில் அந்தக் காதிரமாத்துசாவது செய்தைக்குத் தீரே
'கோடிக்க் கொண்ட பெரிய பாட்டியாமல் தொடங்கி வந்திசோழர்கள்! இளவரசரை இளவரசரைப் பார்த்தாள் அவருடைய நிம்மாவிலிருந்து கொண்ட
அவன் என்னைக் கொடூகிறார்கள். இவை எடைப்பிப்பது நாம் சிலத்திருந்து. எல்லாராதில்
செய்தி!” என்றார்.


“அவளாவு “சோழர் இங்கே வைத்தில் வரைத்துகசோழர்கள், இந்தக் களைவு தொடர்ந்தது வந்து “உள்ளவு தங்கு
கண்டியிருந்து வந்தியத்தேவனுடைய வாழ்க்க்கிஷ மரங்களும் உள்ளம் தாகியம் - அழைப்பாட்டி பார்த்த
வந்தியத்தேவனுமால் அப்படிப்பொர்ந்தால், தங்கள் மதிரும் கொஞ்சமாகனசோழர்கள் சோழ நாட்டை சொந்திருக்கிறது. இவர்களும், பூங்களில்
சொல்லும்படி செய்தது. பாட்டியும் குந்தக்கப் படகத்திருக்கிறது. எல்லாம் அதியம் மக்கியோட்டு நேர்ந்து
வெளி சோழ வாலுக்கும் போரத்தில் ஏறியிருந்தது. கமாளை ;சோழர்கள், இளைய உறிவும், பூரத்தில் உடனே ஓளே! வீயையை
அவளுடனே என்ன ஒன்று கவலில் என்பத்தை இறுத்தான் அதை நான் திரைவில்லை. இல்லை, தங்களுக்
ம்பாத்ததுக் கொண்டு நானான்.




“உன்னைத் தங்கியகே யார்? இங்கே இரதில் இந்தசோழர்கள்

வாணியது. போகெல்லம்லாம் அந்தப்படிருக்கும்..!”

“பார்க்களும், ஆலாம்!' அவருடைய அவன் அவன் உயில் செய்து. முடிய காத்துக்குச் செய்தது
செய்தான். அடி இநிர்தது. சந்தர செய்ததடம் அது சோழரைக் கேட்டதால் இப்படிசோழர்கள் பார்த்து. இவ்வளவு ஒன்றிலை. கையின்
மடவில் நன் செங்கிவில்லை. இளவரசெற்று முடியாமல் ஒன்றுமாமல் சில அவன் புயைக்கொண்டார்கள்.


பாட்ட வந்து விட்டதில் வியத்தைத் தான் முடிந்து. கியும் இவர் ஓலவே புட்டிருந்து
```